### Implementing FunkSVD - Solution

In this notebook we will take a look at writing our own function that performs FunkSVD, which will follow the steps you saw in the previous video.  If you find that you aren't ready to tackle this task on your own, feel free to skip to the following video where you can watch as I walk through the steps.

To test our algorithm, we will run it on the subset of the data you worked with earlier.  Run the cell below to get started.

In [31]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import sparse
import svd_tests as t
%matplotlib inline

# Read in the datasets
movies = pd.read_csv('movies_clean.csv')
reviews = pd.read_csv('reviews_clean.csv')

del movies['Unnamed: 0']
del reviews['Unnamed: 0']

# Create user-by-item matrix
user_items = reviews[['user_id', 'movie_id', 'rating', 'timestamp']]
user_by_movie = user_items.groupby(['user_id', 'movie_id'])['rating'].max().unstack()

# Create data subset
user_movie_subset = user_by_movie[[75314,  68646, 99685]].dropna(axis=0)
ratings_mat = np.matrix(user_movie_subset)
print(ratings_mat)

[[ 7. 10.  8.]
 [ 6. 10.  7.]
 [ 8.  9.  8.]
 [ 8. 10. 10.]
 [ 9.  9.  9.]
 [ 8.  9.  9.]]


In [33]:
movies.shape

(35479, 35)

In [35]:
reviews.shape

(100001, 5)

In [36]:
user_by_movie.shape

(8022, 13850)

`1.` You will use the **user_movie_subset** matrix to show that your FunkSVD algorithm will converge.  In the below cell, use the comments and document string to assist you as you complete writing your own function to complete FunkSVD.  You may also want to try to complete the funtion on your own without the assistance of comments.  You may feel free to remove and add to the function in any way that gets you a working solution! 

**Notice:** There isn't a sigma matrix in this version of matrix factorization.

In [20]:
def FunkSVD(ratings_mat, latent_features=4, learning_rate=0.0001, iters=100):
    '''
    This function performs matrix factorization using a basic form of FunkSVD with no regularization
    
    INPUT:
    ratings_mat - (numpy array) a matrix with users as rows, movies as columns, and ratings as values
    latent_features - (int) the number of latent features used
    learning_rate - (float) the learning rate 
    iters - (int) the number of iterations
    
    OUTPUT:
    user_mat - (numpy array) a user by latent feature matrix
    movie_mat - (numpy array) a latent feature by movie matrix
    '''
    
    # Set up useful values to be used through the rest of the function
    n_users = ratings_mat.shape[0]
    n_movies = ratings_mat.shape[1]
    n_nan = np.sum(np.sum(np.isnan(ratings_mat), axis=0), axis=1)[0,0]
    num_ratings = n_users*n_movies - n_nan
    # Alternative:
    # num_ratings = np.count_nonzero(~np.isnan(ratings_mat))
    
    # Initialize the user and movie matrices with random values
    # https://numpy.org/doc/stable/reference/random/generated/numpy.random.rand.html
    user_mat = np.random.rand(n_users,latent_features)
    movie_mat = np.random.rand(latent_features, n_movies)
    
    # Initialize sse at 0 for first iteration
    sse_accum = 0
    
    print("Optimizaiton Statistics")
    print("Iterations | Mean Squared Error ")
    
    for it in range(iters):
        # Update our sse
        old_sse = sse_accum
        sse_accum = 0
        
        for i in range(n_users):
            for j in range(n_movies):
                r_ij = ratings_mat[i,j]
                if not np.isnan(r_ij):
                    u_i = user_mat[i,:]
                    v_j = movie_mat[:,j]
                    # Compute the error as the actual minus the dot product
                    # of the user and movie latent features
                    e_ij = (r_ij - np.dot(u_i,v_j))**2
                    # Keep track of the sum of squared errors for the matrix
                    sse_accum += e_ij                    
                    # Update the values in each matrix in the direction of the gradient
                    # dE/du_i = -2(a_ij - u_i*v_j)*v_j
                    # dE/dv_j = -2(a_ij - u_i*v_j)*u_i
                    # x_new <- x_old - alpha * dE/dx
                    u_new = u_i + 2.0*learning_rate*(r_ij - np.dot(u_i,v_j))*v_j
                    v_new = v_j + 2.0*learning_rate*(r_ij - np.dot(u_i,v_j))*u_new
                    user_mat[i,:] = u_new
                    movie_mat[:,j] = v_new

        print(f"{it} | {sse_accum}")

    return user_mat, movie_mat 

`2.` Try out your function on the **user_movie_subset** dataset.  First try 3 latent features, a learning rate of 0.005, and 10 iterations.  When you take the dot product of the resulting U and V matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?

In [21]:
# Use your function with 3 latent features, lr of 0.005 and 10 iterations
user_mat, movie_mat = FunkSVD(ratings_mat=ratings_mat,
                              latent_features=3,
                              learning_rate=0.005,
                              iters=10)

Optimizaiton Statistics
Iterations | Mean Squared Error 
0 | 908.2933277741536
1 | 632.688661950179
2 | 349.6213932965372
3 | 148.9564205968025
4 | 53.43853287844017
5 | 21.252235396322945
6 | 12.617523935110881
7 | 10.461291427242324
8 | 9.85106047356348
9 | 9.611582433318963


In [22]:
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[ 7.58093691  9.16365197  8.46181382]
 [ 7.07800734  8.56994439  7.56590017]
 [ 7.46303981  9.0111772   8.40785591]
 [ 8.53221956 10.31502179  9.34586598]
 [ 8.12259308  9.81721833  8.89875982]
 [ 7.76777163  9.38681644  8.48406098]]
[[ 7. 10.  8.]
 [ 6. 10.  7.]
 [ 8.  9.  8.]
 [ 8. 10. 10.]
 [ 9.  9.  9.]
 [ 8.  9.  9.]]


_**Task: compare the predicted vs actual ratings and write your findings here**_

`3.` Let's try out the function again on the **user_movie_subset** dataset.  This time we will again use 3 latent features and a learning rate of 0.005.  However, let's bump up the number of iterations to 300.  When you take the dot product of the resulting U and V matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?  What do you notice about the `mean squared error` at the end of each training iteration?

In [23]:
#use your function with 3 latent features, lr of 0.005 and 300 iterations
user_mat, movie_mat = FunkSVD(ratings_mat=ratings_mat,
                              latent_features=3,
                              learning_rate=0.005,
                              iters=300)

Optimizaiton Statistics
Iterations | Mean Squared Error 
0 | 982.9294445147943
1 | 725.3239802034384
2 | 432.6206599833486
3 | 198.1108852757119
4 | 72.40283087056682
5 | 26.161688112480093
6 | 13.321420576256218
7 | 10.238289816148885
8 | 9.471403192089301
9 | 9.214078687403543
10 | 9.074754791850333
11 | 8.9679671289896
12 | 8.872205517671059
13 | 8.780519574912647
14 | 8.69008958204111
15 | 8.5996172112697
16 | 8.508465807695835
17 | 8.416304519266156
18 | 8.322945139243085
19 | 8.228265869945036
20 | 8.132176364678838
21 | 8.034602372591658
22 | 7.935479522021477
23 | 7.834751242450377
24 | 7.732368478385877
25 | 7.628290117713156
26 | 7.522483654192936
27 | 7.414925879482509
28 | 7.3056035241114134
29 | 7.194513820318364
30 | 7.0816649808692596
31 | 6.967076594875442
32 | 6.850779942538773
33 | 6.732818229460717
34 | 6.613246739347958
35 | 6.492132902332649
36 | 6.369556274960837
37 | 6.245608427255456
38 | 6.1203927321163025
39 | 5.994024052635174
40 | 5.866628323629062
41 | 5.73

In [37]:
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[ 7.1525123  10.00047635  7.99950524]
 [ 5.99895315  9.99939558  7.00129872]
 [ 8.00102569  9.00033239  7.99898774]
 [ 8.00659736 10.00259352  9.99298426]
 [ 9.00346307  9.00127494  8.99627831]
 [ 7.99033977  8.99603392  9.01028025]]
[[ 7. 10.  8.]
 [ 6. 10.  7.]
 [ 8.  9.  8.]
 [ 8. 10. 10.]
 [ 9.  9.  9.]
 [ 8.  9.  9.]]


_**Task: compare the predicted vs actual ratings and write your findings here**_

The last time we placed an **np.nan** value into this matrix the entire svd algorithm in python broke.  Let's see if that is still the case using your FunkSVD function.  In the below cell, I have placed a nan into the first cell of your numpy array.  

`4.` Use 3 latent features, a learning rate of 0.005, and 450 iterations.  Are you able to run your SVD without it breaking (something that was not true about the python built in)?  Do you get a prediction for the nan value?  What is your prediction for the missing value? Use the cells below to answer these questions.

In [38]:
ratings_mat[0, 0] = np.nan
ratings_mat

matrix([[nan, 10.,  8.],
        [ 6., 10.,  7.],
        [ 8.,  9.,  8.],
        [ 8., 10., 10.],
        [ 9.,  9.,  9.],
        [ 8.,  9.,  9.]])

In [39]:
# run SVD on the matrix with the missing value
# use your function with 3 latent features, lr of 0.005 and 450 iterations
user_mat, movie_mat = FunkSVD(ratings_mat=ratings_mat,
                              latent_features=3,
                              learning_rate=0.005,
                              iters=450)

Optimizaiton Statistics
Iterations | Mean Squared Error 
0 | 1064.6388613732595
1 | 858.9601176712201
2 | 594.4411081927411
3 | 332.2771164712286
4 | 148.1164040030606
5 | 57.49402053656463
6 | 24.012784814432436
7 | 13.42717855371512
8 | 10.086538796109158
9 | 8.88122083074736
10 | 8.34531819136841
11 | 8.050495216744338
12 | 7.855972103494983
13 | 7.707784731969549
14 | 7.582322504341723
15 | 7.468273861819386
16 | 7.359895822967392
17 | 7.254158146703718
18 | 7.149418382790521
19 | 7.044770400376678
20 | 6.939711020639231
21 | 6.833964219388979
22 | 6.727386143265955
23 | 6.619912734783831
24 | 6.511530339151776
25 | 6.4022589368933485
26 | 6.292142410898844
27 | 6.181242769376034
28 | 6.069636602019093
29 | 5.957412792884861
30 | 5.8446709310086
31 | 5.731520096721857
32 | 5.618077837685297
33 | 5.5044692276252185
34 | 5.3908259471258475
35 | 5.277285353412771
36 | 5.163989522759522
37 | 5.051084259489859
38 | 4.938718072137486
39 | 4.827041121658619
40 | 4.716204149526087
41 | 4.6

In [41]:
preds = np.dot(user_mat, movie_mat)
print("The predicted value for the missing rating is: {}".format(preds[0,0]))
print()
print("That's right! You just predicted a rating for a user-movie pair that was never rated!")
print("But if you look in the original matrix, this was actually a value of 7. Not bad!")

The predicted value for the missing rating is: 6.799278015389177

That's right! You just predicted a rating for a user-movie pair that was never rated!
But if you look in the original matrix, this was actually a value of 7. Not bad!


In [42]:
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[ 6.79927802  9.99644121  8.00370784]
 [ 5.99733358  9.99990057  7.00172179]
 [ 7.89279109  8.95792566  8.11035275]
 [ 8.09221844 10.03864715  9.90468108]
 [ 9.02236761  9.00802682  8.97747777]
 [ 7.9786558   8.99231456  9.02113112]]
[[nan 10.  8.]
 [ 6. 10.  7.]
 [ 8.  9.  8.]
 [ 8. 10. 10.]
 [ 9.  9.  9.]
 [ 8.  9.  9.]]


Now let's extend this to a more realistic example. Unfortunately, running this function on your entire user-movie matrix is still not something you likely want to do on your local machine.  However, we can see how well this example extends to 1000 users.  In the above portion, you were using a very small subset of data with no missing values.

`5.` Given the size of this matrix, this will take quite a bit of time.  Consider the following hyperparameters: 3 latent features, 0.005 learning rate, and 500 iterations.  Grab a snack, take a walk, and this should be done running in a bit.

In [28]:
# Setting up a matrix of the first 1000 users with movie ratings
first_1000_users = np.matrix(user_by_movie.head(1000))

# perform funkSVD on the matrix of the top 1000 users
# fit to 1000 users with 3 latent features, lr of 0.005, and 500 iterations
user_mat, movie_mat = FunkSVD(ratings_mat=ratings_mat,
                              latent_features=3,
                              learning_rate=0.005,
                              iters=500)

Optimizaiton Statistics
Iterations | Mean Squared Error 
0 | 1111.888006569856
1 | 952.9657136762671
2 | 722.7373740334392
3 | 456.36587170541975
4 | 231.234177904896
5 | 97.63151055446653
6 | 39.89810943779286
7 | 19.621372381221477
8 | 12.911183881218799
9 | 10.516157162988222
10 | 9.521432866129274
11 | 9.037051267246234
12 | 8.766768933284743
13 | 8.597081223781716
14 | 8.4782705247685
15 | 8.386305220513377
16 | 8.308779724489284
17 | 8.238995183522999
18 | 8.173210108014203
19 | 8.109274836977885
20 | 8.045921203847286
21 | 7.982380357991922
22 | 7.918171780680401
23 | 7.852984449118896
24 | 7.786608848140698
25 | 7.718897605094741
26 | 7.649742497918329
27 | 7.579060951326545
28 | 7.506788090217061
29 | 7.432872077561797
30 | 7.357271409165852
31 | 7.279953382875134
32 | 7.20089327726076
33 | 7.120073960972596
34 | 7.037485763734212
35 | 6.953126504971337
36 | 6.867001614715778
37 | 6.779124304478444
38 | 6.689515759584451
39 | 6.598205332769651
40 | 6.505230723884514
41 | 6.410

`6.` Now that you have a set of predictions for each user-movie pair.  Let's answer a few questions about your results. Provide the correct values to each of the variables below, and check your solutions using the tests below.

In [29]:
# How many actual ratings exist in first_1000_users
num_ratings = np.count_nonzero(~np.isnan(first_1000_users))
print("The number of actual ratings in the first_1000_users is {}.".format(num_ratings))
print()

# How many ratings did we make for user-movie pairs that didn't have ratings
ratings_for_missing = first_1000_users.shape[0]*first_1000_users.shape[1] - num_ratings
print("The number of ratings made for user-movie pairs that didn't have ratings is {}".format(ratings_for_missing))

The number of actual ratings in the first_1000_users is 14287.

The number of ratings made for user-movie pairs that didn't have ratings is 13835713


In [30]:
# Test your results against the solution
assert num_ratings == 14287, "Oops!  The number of actual ratings doesn't quite look right."
assert ratings_for_missing == 13835713, "Oops!  The number of movie-user pairs that you made ratings for that didn't actually have ratings doesn't look right."

# Make sure you made predictions on all the missing user-movie pairs
preds = np.dot(user_mat, movie_mat)
assert np.isnan(preds).sum() == 0
print("Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?")

Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?
